In [20]:
from PIL import Image, ImageDraw
import numpy as np
import pandas as pd
import cv2
import fnmatch
import os
import glob

In [21]:
df = pd.read_csv('data.txt', sep=" ", header=None)
df.columns = ["ID", "background_tissue", "class",
              "severity", "x", "y", "rad"]
df.index = df.index + 1
df = df.dropna()

In [23]:
df['severity'].value_counts()

B    68
M    51
Name: severity, dtype: int64

In [24]:
im = Image.open('all-mias/mdb001.pgm') # PILLOW image processing

In [34]:
im.getpixel( (535,425) )

155

In [25]:
last = 'start'

In [26]:
def img_draw(image, x, y, r, c):
    if c=='M':
        color = 'red'
    else:
        color = 'blue'
    draw = ImageDraw.Draw(image)
    leftUpPoint = (x-r, y-r)
    rightDownPoint = (x+r, y+r)
    twoPointList = [leftUpPoint, rightDownPoint]
    draw.ellipse(twoPointList, outline=color, width=5)

In [27]:
for index, row in df.iterrows():
    path = 'all-mias/{name}.pgm'.format(name=row['ID'])
    if last != path:
        im = Image.open(path).convert("RGB")
    x = int(row['x'])
    y = int(row['y'])
    r = int(row['rad'])
    c = row['severity']
    img_draw(im, x, 1024 - y, r, c)
    im.save("marked/{}.png".format(row['ID']))
    last = path

In [28]:
mean_B = []
var_B = []
mean_M = []
var_M = []
for index, row in df.iterrows():
    path = 'all-mias/{name}.pgm'.format(name=row['ID'])
    im = cv2.imread('all-mias/mdb001.pgm')
    h,w,d = im.shape
    x = int(row['x'])
    y = int(row['y'])
    r = int(row['rad'])
    c = row['severity']
    mask = np.zeros((h,w), np.uint8)
    cv2.circle(mask,(x,1024-y),r ,1,thickness=-1)
    masked_data = cv2.bitwise_and(im, im, mask=mask)
    if (c == 'M'):
        mean_M.append(cv2.meanStdDev(im, mask = mask)[0][0])
        var_M.append(np.power(cv2.meanStdDev(im, mask = mask)[1][0], 2))
    else:
        mean_B.append(cv2.meanStdDev(im, mask = mask)[0][0])
        var_B.append(np.power(cv2.meanStdDev(im, mask = mask)[1][0], 2))

In [96]:
len(mean_M)

51

In [98]:
len(mean_B)

68

In [70]:
im = cv2.imread('all-mias/mdb001.pgm')

In [71]:
h,w,d = im.shape

In [72]:
mask = np.zeros((h,w), np.uint8)

In [73]:
cv2.circle(mask,(535,1024-425),197,1,thickness=-1)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [74]:
masked_data = cv2.bitwise_and(im, im, mask=mask)

In [75]:
cv2.imshow("masked", masked_data)
cv2.waitKey(0)

-1

In [76]:
cv2.mean(im, mask = mask)

(150.78105081826013, 150.78105081826013, 150.78105081826013, 0.0)

In [79]:
cv2.meanStdDev(im, mask = mask)[0][0]

array([150.78105082])

In [61]:
cv2.imwrite("masked.png", masked_data)

True